In [397]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [398]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [399]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 32

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-2.1500e+00,  5.6513e-01,  4.2002e-01,  1.3731e+00, -6.7743e-01,
          6.1851e-02,  1.8740e+00, -3.9445e-01,  4.4138e-01, -2.8179e-01,
          2.5136e-01, -1.9627e-01, -1.8123e+00, -2.5395e-01, -5.4424e-02,
          1.4009e+00,  1.3132e-03, -3.0845e-02, -4.8093e-02,  1.3133e-01,
         -3.0582e-01, -1.1654e+00, -1.6748e+00, -4.3975e-01,  5.4659e-01,
          5.8848e-01, -1.7453e+00, -2.8481e-01,  1.7544e+00,  7.0527e-01,
          1.1889e+00,  1.0716e+00],
        [-6.5554e-02,  5.0678e-02, -5.2280e-02,  3.7782e-01,  1.4964e+00,
         -9.3869e-01, -9.5460e-01, -2.5039e-01, -3.8097e-01, -6.6507e-01,
          4.4919e-01, -1.5467e+00,  3.0892e-01, -2.2186e+00,  5.2985e-01,
          9.0416e-02,  6.1396e-01,  5.1731e-01, -3.5931e+00, -2.6052e-01,
          4.4528e-02,  8.2139e-02, -1.3433e+00, -5.9487e-01,  6.7176e-01,
         -1.5383e+00,  1.3294e+00, -4.1598e-01, -4.6920e-01,  6.8838e-01,
         -1.0551e+00,  1.0263e+00]], grad_fn=<EmbeddingBackward0>)


In [400]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [401]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 8,  7,  8,  8,  2,  8,  0,  3,  6, 10],
        [ 6,  8,  9,  7,  2,  1,  5,  4,  6,  2],
        [ 1,  6, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  3,  7,  7,  2,  8,  4,  6,  9,  3],
        [ 9,  9,  6,  3,  8,  4, 10, 10, 10, 10],
        [ 5,  9,  0,  4, 10, 10, 10, 10, 10, 10],
        [ 9,  1,  5,  0,  8,  5,  2,  1,  4,  2],
        [ 8,  1, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 7,  2,  1, 10, 10, 10, 10, 10, 10, 10],
        [ 9,  1,  1,  5,  7,  7,  1,  2, 10, 10],
        [ 3,  8,  3,  5,  9,  3,  5,  0,  9,  3],
        [ 0,  5, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 3,  0,  9,  5,  8,  7,  3,  8,  6,  1],
        [ 7,  3,  6,  4,  7,  6,  1, 10, 10, 10],
        [ 2,  5,  9,  1,  7,  3,  0, 10, 10, 10],
        [ 6,  5,  0,  3, 10, 10, 10, 10, 10, 10],
        [ 8,  1,  6, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  3,  7,  7,  2,  8,  9, 10, 10, 10],
        [ 9,  1,  7, 10, 10, 10, 10, 10, 10, 10],
        [ 6,  8,  5,  4,  3,  7,  9, 10, 10, 10],


In [402]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW,
                    use_pre_lnorm = True)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [403]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [404]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 318.04it/s]

[3.335501567649841, 1.9809635179519653, 1.5787576435089112, 1.6106122561454772, 1.2086370057582856, 1.1334184087753296, 1.1664608815670015, 1.0315802058696746, 1.0191582596302033, 0.9059406329751015]
[0.0088, 0.0202, 0.0321, 0.03, 0.0457, 0.0469, 0.0452, 0.0475, 0.0499, 0.0602]
[0.5227992573176059, 0.5721987469036864, 0.587727935813275, 0.5862917334888468, 0.6003553206997084, 0.6030944744131798, 0.6019530680658796, 0.6059683535073648, 0.6050560979163631, 0.6104922091160623]

[3.516962059020996, 3.60521293258667, 1.1186264715194703, 0.9379270858764649, 3.1093127269744874, 0.5991635065078735, 2.251642430305481, 1.5489190793037415, 0.8688369910717011, 1.4359000940322877]
[0.003, 0.002, 0.042, 0.034, 0.004, 0.056, 0.007, 0.017, 0.056, 0.031]
[0.5282434402332361, 0.5259657434402333, 0.6067784256559767, 0.6144314868804664, 0.5588556851311953, 0.6221756559766763, 0.579719387755102, 0.5870991253644315, 0.6135204081632653, 0.5966654518950437]
